```console
(base) root@notebook-7bc42cfb-81fb-47ef-a08c-c0cdca223bba-0:~/.cache# lscpu
Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   52 bits physical, 57 bits virtual
CPU(s):                          144
On-line CPU(s) list:             0-143
Thread(s) per core:              2
Core(s) per socket:              36
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           106
Model name:                      Intel(R) Xeon(R) Platinum 8352V CPU @ 2.10GHz
Stepping:                        6
CPU MHz:                         804.863
CPU max MHz:                     3500.0000
CPU min MHz:                     800.0000
BogoMIPS:                        4200.00
Virtualization:                  VT-x
L1d cache:                       3.4 MiB
L1i cache:                       2.3 MiB
L2 cache:                        90 MiB
L3 cache:                        108 MiB
NUMA node0 CPU(s):               0-35,72-107
NUMA node1 CPU(s):               36-71,108-143
Vulnerability Itlb multihit:     Not affected
Vulnerability L1tf:              Not affected
Vulnerability Mds:               Not affected
Vulnerability Meltdown:          Not affected
Vulnerability Spec store bypass: Mitigation; Speculative Store Bypass disabled via prctl and seccomp
Vulnerability Spectre v1:        Mitigation; usercopy/swapgs barriers and __user pointer sanitization
Vulnerability Spectre v2:        Mitigation; Enhanced IBRS, IBPB conditional, RSB filling
Vulnerability Srbds:             Not affected
Vulnerability Tsx async abort:   Not affected
Flags:                           fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good 
                                 nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid dca sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave
                                  avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb cat_l3 invpcid_single ssbd mba ibrs ibpb stibp ibrs_enhanced tpr_shadow vnmi flexpriority ept vpid ept_ad fsgsbase tsc_adjust bmi1 avx2 sm
                                 ep bmi2 erms invpcid cqm rdt_a avx512f avx512dq rdseed adx smap avx512ifma clflushopt clwb intel_pt avx512cd sha_ni avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves cqm_llc cqm_occup_llc cqm_mbm_to
                                 tal cqm_mbm_local wbnoinvd dtherm ida arat pln pts hwp hwp_act_window hwp_epp hwp_pkg_req avx512vbmi umip pku ospke avx512_vbmi2 gfni vaes vpclmulqdq avx512_vnni avx512_bitalg tme avx512_vpopcntdq 
                                 rdpid md_clear pconfig flush_l1d arch_capabilities
```

In [ ]:
Sat Jun 29 03:55:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:15:00.0 Off |                  Off |
| 30%   47C    P2             321W / 450W |  12615MiB / 24564MiB |     99%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce RTX 4090        Off | 00000000:18:00.0 Off |                  Off |
| 30%   36C    P2              56W / 450W |   5663MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
+---------------------------------------------------------------------------------------+

In [1]:
# 导入必要的库
from datasets import load_dataset, DatasetDict, Audio
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor, AutoModelForSpeechSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model, PeftModel ,PeftConfig
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# 全局参数设置
base_path = "/root/dataDisk/hf/hub"
model_name_or_path = base_path + "/models/whisper-large-v2"
model_dir = "models/whisper-large-v2-asr-int8"
language = "Chinese (China)"
language_abbr = "zh-CN"
task = "transcribe"
dataset_path = base_path +"/datasets/common_voice_11_0/"
batch_size = 64

# 加载数据集
common_voice = DatasetDict()
common_voice["train"] = load_dataset(dataset_path+"common_voice_11_0.py", language_abbr, split="train", cache_dir=dataset_path, trust_remote_code=True)
common_voice["validation"] = load_dataset(dataset_path+"common_voice_11_0.py", language_abbr, split="validation", cache_dir=dataset_path,trust_remote_code=True)

print("数据集加载完成")

# 数据预处理
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
print("1")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
print("2")
processor = AutoProcessor.from_pretrained(model_name_or_path, language=language, task=task)
print("3")


# 移除数据集中不必要的字段
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print("数据预处理完成")

# 数据预处理函数
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

# 应用数据预处理
tokenized_common_voice = common_voice.map(prepare_dataset,num_proc=2)

print("数据预处理应用完成")

# 数据整理器定义
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

print("数据整理器实例化完成")

# 加载和配置模型
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")
model = prepare_model_for_int8_training(model)

print("模型加载和配置完成")

# LoRA配置
config = LoraConfig(
    r=4, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none"
)
peft_model = get_peft_model(model, config)

print("LoRA配置完成")

# 训练设置
training_args = Seq2SeqTrainingArguments(
    output_dir=model_dir, per_device_train_batch_size=batch_size, learning_rate=1e-3,
    num_train_epochs=1, evaluation_strategy="epoch", per_device_eval_batch_size=batch_size,
    generation_max_length=128, logging_steps=10, remove_unused_columns=False, label_names=["labels"]
)

trainer = Seq2SeqTrainer(
    args=training_args, model=peft_model, train_dataset=tokenized_common_voice["train"],
    eval_dataset=tokenized_common_voice["validation"], data_collator=data_collator, tokenizer=processor.feature_extractor
)
peft_model.config.use_cache = False
trainer.train()
trainer.save_model(model_dir)

print("模型训练和保存完成")

peft_model.eval()

# 推理（可能需要重启 Notebook 来清除内存）
peft_config = PeftConfig.from_pretrained(model_dir)
base_model = AutoModelForSpeechSeq2Seq.from_pretrained(peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto")
peft_model = PeftModel.from_pretrained(base_model, model_dir)
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = AutoProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor

# 语音识别管道
test_audio = "data/audio/test_zh.flac"
from transformers import AutomaticSpeechRecognitionPipeline
pipeline = AutomaticSpeechRecognitionPipeline(model=peft_model, tokenizer=tokenizer, feature_extractor=feature_extractor)
with torch.cuda.amp.autocast():
    text = pipeline(test_audio, max_new_tokens=255)["text"]
print(text)

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


数据集加载完成
1


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2
3
数据预处理完成


Map (num_proc=2): 100%|██████████| 10581/10581 [16:55<00:00, 10.42 examples/s]  


...

模型加载和配置完成
LoRA配置完成


/root/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/root/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,0.327500,0.384317


模型训练和保存完成


NameError: name 'PeftConfig' is not defined

### 运行结果
```PLAINTEXT
/root/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
数据集加载完成
1
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2
3
数据预处理完成
Map (num_proc=2): 100%|██████████| 29056/29056 [43:52<00:00, 11.04 examples/s]   
Map (num_proc=2): 100%|██████████| 10581/10581 [16:55<00:00, 10.42 examples/s]  
数据预处理应用完成
数据整理器实例化完成
/root/.local/lib/python3.10/site-packages/peft/utils/other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
模型加载和配置完成
LoRA配置完成
/root/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/root/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
 [454/454 2:42:14, Epoch 1/1]
Epoch	Training Loss	Validation Loss
1	0.327500	0.384317
模型训练和保存完成
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[1], line 107
    104 peft_model.eval()
    106 # 推理（可能需要重启 Notebook 来清除内存）
--> 107 peft_config = PeftConfig.from_pretrained(model_dir)
    108 base_model = AutoModelForSpeechSeq2Seq.from_pretrained(peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto")
    109 peft_model = PeftModel.from_pretrained(base_model, model_dir)

NameError: name 'PeftConfig' is not defined
```